Check Tide Map
==============

This ({nb-download}`notebook <Check-Tide-Map.ipynb>`) demonstrates checking if given points are within a tide model domain

```{important}
*Need to download tide model prior to running this notebook.*  
```

OTIS format tidal solutions provided by Oregon State University and ESR  
- [http://volkov.oce.orst.edu/tides/region.html](http://volkov.oce.orst.edu/tides/region.html) 
- [https://www.esr.org/research/polar-tide-models/list-of-polar-tide-models/](https://www.esr.org/research/polar-tide-models/list-of-polar-tide-models/)
- [ftp://ftp.esr.org/pub/datasets/tmd/](ftp://ftp.esr.org/pub/datasets/tmd/)  

Global Tide Model (GOT) solutions provided by Richard Ray at GSFC  
- [https://earth.gsfc.nasa.gov/geo/data/ocean-tide-models](https://earth.gsfc.nasa.gov/geo/data/ocean-tide-models)

Finite Element Solution (FES) provided by AVISO  
- [https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/global-tide-fes.html](https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/global-tide-fes.html
)

## Python Dependencies
 - [h5netcdf: Pythonic interface to netCDF4 via h5py](https://h5netcdf.org/)
 - [ipyleaflet: Jupyter / Leaflet bridge enabling interactive maps](https://github.com/jupyter-widgets/ipyleaflet)  
 - [matplotlib: Python 2D plotting library](https://matplotlib.org/)  
 - [numpy: Scientific Computing Tools For Python](https://www.numpy.org)  
 - [pyproj: Python interface to PROJ library](https://pypi.org/project/pyproj/)  
 - [scipy: Scientific Tools for Python](https://www.scipy.org/)
 - [xarray: N-D labeled arrays and datasets in Python](https://docs.xarray.dev/en/stable/) 

## Program Dependencies

- `io.model.py`: retrieves tide model parameters for named tide models
- `io.OTIS.py`: extract tidal harmonic constants from OTIS tide models  
- `io.ATLAS.py`: extract tidal harmonic constants from netcdf models  
- `io.GOT.py`: extract tidal harmonic constants from GSFC GOT models  
- `io.FES.py`: extract tidal harmonic constants from FES tide models  

```{note}
This notebook uses Jupyter widgets to set parameters for calculating the tidal maps.  
```

## Load modules

In [ ]:
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
import IPython.display

# import tide programs
import pyTMD.io
import pyTMD.tools
# autoreload
%load_ext autoreload
%autoreload 2

## Set parameters for program

- Model directory  
- Tide model  

In [ ]:
# available model list
model_list = sorted(pyTMD.io.model.ocean_elevation())
# display widgets for setting directory and model
TMDwidgets = pyTMD.tools.widgets()
TMDwidgets.model.options = model_list
TMDwidgets.model.value = 'GOT4.10_nc'
TMDwidgets.VBox([
    TMDwidgets.directory,
    TMDwidgets.model,
])

## Select location to check

In [ ]:
# default coordinates to use
LAT,LON = (32.86710263,-117.25750387)
m = pyTMD.tools.leaflet(center=(LAT,LON), zoom=12,
    zoom_control=True, marker_control=True,
    attribution=False)
# show map
m.map

## Read and create mask of valid model values

In [ ]:
# get model parameters
model = pyTMD.io.model(TMDwidgets.directory.value,
   ).from_database(TMDwidgets.model.value)
# open model as dataset
ds = model.open_dataset(group='z', chunks='auto')
mz = np.logical_not(ds[ds.tmd.constituents[0]].real.isnull())

In [ ]:
def update_coordinates(sender):
    # leaflet location
    LAT,LON = np.copy(m.marker.location)
    # verify longitudes
    LON = m.wrap_longitudes(LON)
    # adjust dimensions of input coordinates to be iterable
    LON = np.atleast_1d(LON)
    LAT = np.atleast_1d(LAT)
    # convert to model grid coordinates
    x, y = ds.tmd.transform_as(LON, LAT)
    # update plot coordinates
    m.point.set_xdata(x)
    m.point.set_ydata(y)
    # refresh plot
    IPython.display.display(m.figure)

## Plot mask and selected location

In [ ]:
%matplotlib widget
# check coordinates on tide grid
m.figure,ax = plt.subplots(num=1, figsize=(8.25,5.25))
mz.plot(ax=ax, cmap='gray', add_labels=False, add_colorbar=False)
m.point, = ax.plot([],[],'r*')
update_coordinates(None)
# no ticks on the x and y axes
ax.get_xaxis().set_ticks([])
ax.get_yaxis().set_ticks([])
# stronger linewidth on frame
[i.set_linewidth(2.0) for i in ax.spines.values()]
# adjust subplot within figure
m.figure.tight_layout()
IPython.display.clear_output(wait=True)
m.marker.observe(update_coordinates)